In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *

In [4]:
import re
import codecs
import math

# Data

In [17]:
evalData = getEvalData(2, maxExtraNews=0, maxUsers=100)
bp(evalData.keys(), 5)
log((evalData['meta']))

--> tic: 20.4s | message: Eval data loaded
--> toc total duration: 20.68s | message: Got Twinews evaluation data
{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }
{'usersCount': 15905, 'trainNewsCount': 237150, 'testNewsCount': 71781, 'totalNewsAvailable': 570210, 'trainMeanNewsPerUser': 26.48, 'testMeanNewsPerUser': 7.22, 'trainMinNewsPerUser': 8, 'testMinNewsPerUser': 2, 'trainMaxNewsPerUser': 379, 'testMaxNewsPerUser': 97, 'created': '2020.03.24-14.28.06', 'ranksLength': 1000, 'splitDate': '2017-12-25', 'startDate': '2017-10-01', 'endDate': '2018-01-15', 'id': 2}


In [21]:
userUrls = evalData["testUsers"]
bp(userUrls)

{
  100585750: { http://ow.ly/AcC830hnmmW: 1514578206, http://ow.ly/EHLH30hEyij: 1515693670, http://ow.ly/SiXu30hxfff: 1515004208, http://ow.ly/gpYy30hxhde: 1515074706, http://ow.ly/smGT30hEjUZ: 1515603648, http://ow.ly/vk8f30hEiBM: 1515528060 },
  107115739: 
  {
    http://inthesetimes.com/article/20732/Abortion-Chicago-Midwest-Action-Coalition-women: 1514554634,
    http://politi.co/2mck5Fb: 1515413337,
    http://thehill.com/homenews/campaign/366173-seven-primary-races-to-watch-in-2018: 1514307853,
    http://www.gainesville.com/opinion/20180109/indivisible-gainesville-dont-fund-deceptive-anti-abortion-centers: 1515534540,
    http://www.hawaiinewsnow.com/story/37187709/hawaii-joins-tele-abortion-pilot-amid-larger-debate-about-procedures-safety#.Wk41ifxxFrY.twitter: 1515076860,
    ...,
    https://www.huffingtonpost.com/entry/women-wearing-black-state-of-the-union_us_5a55587ee4b03417e873286a: 1515639892,
    https://www.politico.com/newsletters/illinois-playbook/2018/01/03/lipinsk

In [22]:
# get sentences

# BM25 parameters and intialization

In [18]:
# TODO Do we need to train k,b?
# TODO if not, how to do the ranking for testuser?

In [10]:
f = [] # a list of dict with counting of frequency of each word
tf = {} # term frequency
idf = {} # IDF of each word
k1 = 1.5
b = 0.75 # settled parametre
 

# TF, IDF

In [11]:
def inition(docs):
    D = len(docs)
    avgdl = sum([len(doc)+ 0.0 for doc in docs]) / D # average length of document
    for doc in docs:
        tmp = {}
        for word in doc:
            tmp[word] = tmp.get(word, 0) + 1  # store the frequency of words temporarily
        f.append(tmp)
        for k in tmp.keys():
            tf[k] = tf.get(k, 0) + 1         # term frequency
    for k, v in tf.items():
        idf[k] = math.log(D - v + 0.5) - math.log(v + 0.5)
    return D, avgdl

# Similarity

In [12]:
def sim(doc, index):
    score = 0.0
    for word in doc:
        if word not in f[index]:
            continue
        d = len(document[index])
        score += (idf[word] * f[index][word] * (k1 + 1) / (f[index][word] + k1 * (1 - b + b * d / avgdl)))
    return score

In [13]:
def simall(doc):
    scores = []
    for index in range(D):
            score = sim(doc, index)
            scores.append(score)
    return scores